In [1]:
%matplotlib inline

import matplotlib
import numpy as np
np.set_printoptions(threshold=np.nan)
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import networkx as nx
import scipy.sparse as sparse
import asizeof
n_jobs=22

In [2]:
def simulation(n,m,p,init_Theta,N,r,prob):
    G=nx.barabasi_albert_graph(n, p)
    G=G.to_directed()
    for u,v in G.edges():
        G[u][v]['weight']=np.random.uniform(0,1)
    for i in range(n):
        in_degree=G.in_degree(i,weight='weight')
        for u,v in G.in_edges(i):
            G[u][v]['weight']/=in_degree
    W=nx.adjacency_matrix(G,weight='weight')
    C = np.random.random((m, m))
    C=C*2-1
    C *= np.tri(*C.shape,k=-1)
    C=C+np.transpose(C)+np.eye(m)
    indykatory=[]
    frakcje=[]
    I=np.random.choice(a=[0,1], size=(n,m), replace=True, p=[1-prob, prob])
    indykatory.append(I)
    frakcje.append(np.vstack((range(m),np.sum(I,axis=0)/n)))
    I=I.copy()
    Theta=np.full((n,m), init_Theta)
    Y=np.full((n,1),0)
    for l in range(N):    
        U=W.transpose().dot(I)
        F=U.dot(C)/m
        temp=np.greater_equal(F, Theta)
        for i in np.unique(np.where(temp[:,:]==True)[0]):
            temp1=np.where(temp[i,:]==True)[0]
            #print(i,temp1.shape,l)
            if not np.any(C[temp1[:, None],temp1]<0):
                I[i][temp1]=1
                Y[i]+=1
                Theta[i]=1-(1-init_Theta)**Y[i]
            else:
                if np.random.rand()<r:
                    I[i][temp1]=1
                    Y[i]+=1
                    Theta[i]=1-(1-init_Theta)**Y[i]
        indykatory.append(I)
        frakcje.append(np.vstack((range(m),np.sum(I,axis=0)/n)))
        I=I.copy()
    del temp
    del temp1
    return [frakcje,indykatory]

In [5]:
from joblib import Parallel, delayed
import time
def text_progessbar(seq, total=None):
    step = 1
    tick = time.time()
    while True:
        time_diff = time.time()-tick
        avg_speed = time_diff/step
        total_str = 'of %n' % total if total else ''
        print('step', step, '%.2f' % time_diff, 'avg: %.2f iter/sec' % avg_speed, total_str)
        step += 1
        yield next(seq)
all_bar_funcs = {
    'txt': lambda args: lambda x: text_progessbar(x, **args),
    'None': lambda args: iter,
}
def ParallelExecutor(use_bar='tqdm', **joblib_args):
    def aprun(bar=use_bar, **tq_args):
        def tmp(op_iter):
            if str(bar) in all_bar_funcs.keys():
                bar_func = all_bar_funcs[str(bar)](tq_args)
            else:
                raise ValueError("Value %s not supported as bar type"%bar)
            return Parallel(**joblib_args)(bar_func(op_iter))
        return tmp
    return aprun

aprun = ParallelExecutor(n_jobs=n_jobs)

In [9]:
results=aprun(bar='txt')(delayed(simulation)(10000,100,3,i,10,0.1,0.2) for i in np.arange(0.01,0.055,0.0005))

step 1 0.00 avg: 0.00 iter/sec 
step 2 0.00 avg: 0.00 iter/sec 
step 3 0.01 avg: 0.00 iter/sec 
step 4 0.01 avg: 0.00 iter/sec 
step 5 0.01 avg: 0.00 iter/sec 
step 6 0.01 avg: 0.00 iter/sec 
step 7 0.02 avg: 0.00 iter/sec 
step 8 0.02 avg: 0.00 iter/sec 
step 9 0.02 avg: 0.00 iter/sec 
step 10 0.02 avg: 0.00 iter/sec 
step 11 0.02 avg: 0.00 iter/sec 
step 12 0.03 avg: 0.00 iter/sec 
step 13 0.03 avg: 0.00 iter/sec 
step 14 0.03 avg: 0.00 iter/sec 
step 15 0.03 avg: 0.00 iter/sec 
step 16 0.03 avg: 0.00 iter/sec 
step 17 0.03 avg: 0.00 iter/sec 
step 18 0.03 avg: 0.00 iter/sec 
step 19 0.03 avg: 0.00 iter/sec 
step 20 0.04 avg: 0.00 iter/sec 
step 21 0.04 avg: 0.00 iter/sec 
step 22 0.04 avg: 0.00 iter/sec 
step 23 0.04 avg: 0.00 iter/sec 
step 24 0.04 avg: 0.00 iter/sec 
step 25 0.04 avg: 0.00 iter/sec 
step 26 0.04 avg: 0.00 iter/sec 
step 27 0.04 avg: 0.00 iter/sec 
step 28 0.05 avg: 0.00 iter/sec 
step 29 0.05 avg: 0.00 iter/sec 
step 30 0.05 avg: 0.00 iter/sec 
step 31 0.05 avg: 0

In [11]:
asizeof.asizeof(results)/1024/1024/1024

0.0002509653568267822